In [1]:
import json 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler
from funcs import fit_ar_garch, reconstruct_residuals, get_imbalance_price, fit_sample_copula
from copulas.univariate import GaussianKDE, BetaUnivariate, GaussianUnivariate, GammaUnivariate, LogLaplace, StudentTUnivariate
from fetch_da_prices import load_and_process_boiler_data, get_day_ahead_prices
%load_ext autoreload
%autoreload 2

In [2]:
start = pd.Timestamp('20250304', tz='UTC')
end = pd.Timestamp('20250504', tz='UTC')

In [3]:
capacity_up_hat = pd.read_parquet('/home/alqua/git/afrr-price-forecast-engine/data/results/afrr_price_forecast_UP.parquet')
da_prices_hat = load_and_process_boiler_data("/home/alqua/data/boiler_data/daily_data")

da_prices = get_day_ahead_prices(start, end, price_area="DK_1")
da_prices_hat = da_prices_hat.loc[start:end]
pic_prices = get_imbalance_price(start_date=start, end_date=end, price_area="DK1")
capacity_up_hat.index = capacity_up_hat.index.tz_localize('UTC')
pic_prices.index = pic_prices.index.rename('time_utc')

pic_prices.index= pd.to_datetime(pic_prices.index, utc=True)
capacity_up_hat = capacity_up_hat.loc[start:end]

In [4]:
da_prices_df = da_prices_hat.join(da_prices, on="time_utc")
da_prices_df["da_price"] = da_prices_df["da_price"].ffill()
da_prices_df["da_resid"] = da_prices_df["da_preds"] - da_prices_df["da_price"]


In [5]:
da_prices_df["da_resid"] = da_prices_df["da_resid"].diff(1)

In [ ]:
data = pic_prices.join(capacity_up_hat, on='time_utc' )
data = pic_prices.join(capacity_up_hat, on='time_utc' )
data['pic_up_resid'] =  data['aFRRVWAUpEUR'] - data['SpotPriceEUR']

In [7]:
data = data.join(da_prices_df, on='time_utc', how='inner')

In [8]:
data = data.ffill()

In [18]:
#ar_params_cap_up, garch_params_cap_up, std_res_cap_up, cond_vol_up = fit_ar_garch(data['cap_up_resid'], ar_order=1, garch_p=1, garch_q=1)
#ar_params_pic_up, garch_params_pic_up, std_res_cap_pic_up, cond_vol_pic_up = fit_ar_garch(data['pic_up_resid'], ar_order=1, garch_p=1, garch_q=1)
#ar_params_da, garch_params_da, std_res_da, cond_vol_da = fit_ar_garch(data['da_resid'], ar_order=1, garch_p=1, garch_q=1)

#data['cap_up_resid_std'] = std_res_cap_up
#data['pic_up_resid_std'] = std_res_cap_pic_up
#data['da_resid_std'] = std_res_cap_pic_up

In [10]:
from copulas.multivariate import GaussianMultivariate
import numpy as np

dist = GaussianMultivariate(distribution={"cap_up_resid": GaussianKDE, 
                                         "pic_up_resid": GaussianKDE, 
                                         "da_resid": GaussianKDE})

dist.fit(data[["cap_up_resid", "pic_up_resid", "da_resid"]])

samples = dist.sample(10000)

In [11]:
def sample_quantile(samples, dist, target_quantile):
    cdf_values = samples.apply(dist.cumulative_distribution, axis=1)
    idx_closest = (cdf_values - target_quantile).abs().idxmin()
    quantile_sample = samples.loc[idx_closest]
    
    return quantile_sample
    

In [12]:
joint_quantile_q90 = sample_quantile(samples, dist, 0.9)
joint_quantile_q50 = sample_quantile(samples, dist, 0.5)
joint_quantile_q10 = sample_quantile(samples, dist, 0.1)

In [13]:
data['cap_up_resid_q90'] = joint_quantile_q90['cap_up_resid']
data['cap_up_resid_q50'] = joint_quantile_q50['cap_up_resid']
data['cap_up_resid_q10'] = joint_quantile_q10['cap_up_resid']

In [14]:
data['opt_afrr_up_cap_price_q90'] = data['opt_afrr_up_cap_price'] + data['cap_up_resid_q90']
data['opt_afrr_up_cap_price_q50'] = data['opt_afrr_up_cap_price'] + data['cap_up_resid_q50']
data['opt_afrr_up_cap_price_q10'] = data['opt_afrr_up_cap_price'] + data['cap_up_resid_q10']


In [ ]:
da_q90_cond_cap_q90 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q90'].quantile(0.9)]['da_resid'].quantile(0.9)
da_q50_cond_cap_q90 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q90'].quantile(0.9)]['da_resid'].quantile(0.5)
da_q10_cond_cap_q90 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q90'].quantile(0.9)]['da_resid'].quantile(0.1)

da_q90_cond_cap_q50 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q50'].quantile(0.5)]['da_resid'].quantile(0.9)
da_q50_cond_cap_q50 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q50'].quantile(0.5)]['da_resid'].quantile(0.5)
da_q10_cond_cap_q50 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q50'].quantile(0.5)]['da_resid'].quantile(0.1)

da_q90_cond_cap_q10 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q10'].quantile(0.1)]['da_resid'].quantile(0.9)
da_q50_cond_cap_q10 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q10'].quantile(0.1)]['da_resid'].quantile(0.5)
da_q10_cond_cap_q10 = data[data['opt_afrr_up_cap_price'] >= data['opt_afrr_up_cap_price_q10'].quantile(0.1)]['da_resid'].quantile(0.1)


data['da_q90_cond_cap_q10'] = data['da_preds'] + da_q90_cond_cap_q10
data['da_q90_cond_cap_q50'] = data['da_preds'] + da_q90_cond_cap_q50
data['da_q90_cond_cap_q90'] = data['da_preds'] + da_q90_cond_cap_q90
data['da_q50_cond_cap_q10'] = data['da_preds'] + da_q50_cond_cap_q10
data['da_q50_cond_cap_q50'] = data['da_preds'] + da_q50_cond_cap_q50
data['da_q50_cond_cap_q90'] = data['da_preds'] + da_q50_cond_cap_q90
data['da_q10_cond_cap_q10'] = data['da_preds'] + da_q10_cond_cap_q10
data['da_q10_cond_cap_q50'] = data['da_preds'] + da_q10_cond_cap_q50
data['da_q10_cond_cap_q90'] = data['da_preds'] + da_q10_cond_cap_q90


In [16]:
# Brach 1 Picasso
pic_q90_cond_da_q90_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q90_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q90_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.1)
# Brach 2 Picasso
pic_q90_cond_da_q50_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q50_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q50_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.1)
# Brach 3 Picasso
pic_q90_cond_da_q10_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q10_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q10_cond_cap_q90 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.1)
# Brach 4 Picasso
pic_q90_cond_da_q90_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q90_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q90_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.1)
# Brach 5 Picasso
pic_q90_cond_da_q50_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q50_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q50_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.1)
# Brach 6 Picasso
pic_q90_cond_da_q10_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q10_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q10_cond_cap_q50 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.1)
# Brach 7 Picasso
pic_q90_cond_da_q90_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q90_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q90_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q90'].quantile(0.9)]['pic_up_resid'].quantile(0.1)
# Brach 8 Picasso
pic_q90_cond_da_q50_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q50_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q50_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q50'].quantile(0.5)]['pic_up_resid'].quantile(0.1)
# Brach 9 Picasso
pic_q90_cond_da_q10_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.9)
pic_q50_cond_da_q10_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.5)
pic_q10_cond_da_q10_cond_cap_q10 = data[data['da_preds'] >= data['da_q90_cond_cap_q10'].quantile(0.1)]['pic_up_resid'].quantile(0.1)


In [17]:
# Assuming the quantiles are already calculated as per your branches

# Create new columns with specific names based on the quantile combinations
data['pic_q90_cond_da_q90_cond_cap_q90'] = data['da_preds'] + pic_q90_cond_da_q90_cond_cap_q90
data['pic_q50_cond_da_q90_cond_cap_q90'] = data['da_preds'] + pic_q50_cond_da_q90_cond_cap_q90
data['pic_q10_cond_da_q90_cond_cap_q90'] = data['da_preds'] + pic_q10_cond_da_q90_cond_cap_q90

data['pic_q90_cond_da_q50_cond_cap_q90'] = data['da_preds'] + pic_q90_cond_da_q50_cond_cap_q90
data['pic_q50_cond_da_q50_cond_cap_q90'] = data['da_preds'] + pic_q50_cond_da_q50_cond_cap_q90
data['pic_q10_cond_da_q50_cond_cap_q90'] = data['da_preds'] + pic_q10_cond_da_q50_cond_cap_q90

data['pic_q90_cond_da_q10_cond_cap_q90'] = data['da_preds'] + pic_q90_cond_da_q10_cond_cap_q90
data['pic_q50_cond_da_q10_cond_cap_q90'] = data['da_preds'] + pic_q50_cond_da_q10_cond_cap_q90
data['pic_q10_cond_da_q10_cond_cap_q90'] = data['da_preds'] + pic_q10_cond_da_q10_cond_cap_q90

data['pic_q90_cond_da_q90_cond_cap_q50'] = data['da_preds'] + pic_q90_cond_da_q90_cond_cap_q50
data['pic_q50_cond_da_q90_cond_cap_q50'] = data['da_preds'] + pic_q50_cond_da_q90_cond_cap_q50
data['pic_q10_cond_da_q90_cond_cap_q50'] = data['da_preds'] + pic_q10_cond_da_q90_cond_cap_q50

data['pic_q90_cond_da_q50_cond_cap_q50'] = data['da_preds'] + pic_q90_cond_da_q50_cond_cap_q50
data['pic_q50_cond_da_q50_cond_cap_q50'] = data['da_preds'] + pic_q50_cond_da_q50_cond_cap_q50
data['pic_q10_cond_da_q50_cond_cap_q50'] = data['da_preds'] + pic_q10_cond_da_q50_cond_cap_q50

data['pic_q90_cond_da_q10_cond_cap_q50'] = data['da_preds'] + pic_q90_cond_da_q10_cond_cap_q50
data['pic_q50_cond_da_q10_cond_cap_q50'] = data['da_preds'] + pic_q50_cond_da_q10_cond_cap_q50
data['pic_q10_cond_da_q10_cond_cap_q50'] = data['da_preds'] + pic_q10_cond_da_q10_cond_cap_q50

data['pic_q90_cond_da_q90_cond_cap_q10'] = data['da_preds'] + pic_q90_cond_da_q90_cond_cap_q10
data['pic_q50_cond_da_q90_cond_cap_q10'] = data['da_preds'] + pic_q50_cond_da_q90_cond_cap_q10
data['pic_q10_cond_da_q90_cond_cap_q10'] = data['da_preds'] + pic_q10_cond_da_q90_cond_cap_q10

data['pic_q90_cond_da_q50_cond_cap_q10'] = data['da_preds'] + pic_q90_cond_da_q50_cond_cap_q10
data['pic_q50_cond_da_q50_cond_cap_q10'] = data['da_preds'] + pic_q50_cond_da_q50_cond_cap_q10
data['pic_q10_cond_da_q50_cond_cap_q10'] = data['da_preds'] + pic_q10_cond_da_q50_cond_cap_q10

data['pic_q90_cond_da_q10_cond_cap_q10'] = data['da_preds'] + pic_q90_cond_da_q10_cond_cap_q10
data['pic_q50_cond_da_q10_cond_cap_q10'] = data['da_preds'] + pic_q50_cond_da_q10_cond_cap_q10
data['pic_q10_cond_da_q10_cond_cap_q10'] = data['da_preds'] + pic_q10_cond_da_q10_cond_cap_q10